In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.feature import graycomatrix, graycoprops

from skimage import io, color, feature, exposure
from skimage.util import img_as_ubyte
from sklearn.cluster import KMeans

In [ ]:
##### A. Brightness & Kontras###
def brightness_contrast(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)
    contrast = np.std(gray)
    return brightness, contrast

In [ ]:
######## Blur / Focus Level (Variance of Laplacian)####

def blur_score(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

In [ ]:
########C. Edge Density (Canny Edge Detector)####

def edge_density(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return np.sum(edges > 0) / edges.size

In [ ]:

######## D. Entropy (skimage)####
from skimage.measure import shannon_entropy
def image_entropy(img):
    gray = color.rgb2gray(img)
    return shannon_entropy(gray)

In [ ]:
##### E. GLCM (Gray Level Co-occurrence Matrix) Texture Contrast######

def glcm_contrast(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = img_as_ubyte(gray)
    glcm = graycomatrix(gray, distances=[1], angles=[0], symmetric=True, normed=True)
    return graycoprops(glcm, 'contrast')[0,0]

In [ ]:
######## F. Dominant Color (KMeans)####

def dominant_color(img, k=3):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_flat = img_rgb.reshape(-1, 3)
    km = KMeans(n_clusters=k, n_init=10)
    km.fit(img_flat)
    return km.cluster_centers_[np.argmax(np.bincount(km.labels_))]

In [ ]:

######## Calculate per Image ####
features = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    img = cv2.imread(row['path'])
    if img is None:
        continue
    img = cv2.resize(img, (224,224))
    bright, contrast = brightness_contrast(img)
    blur = blur_score(img)
    edge = edge_density(img)
    entropy = image_entropy(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    glcm_c = glcm_contrast(img)
    dom_col = dominant_color(img)
    
    features.append({
        'label': row['label'],
        'brightness': bright,
        'contrast': contrast,
        'blur': blur,
        'edges': edge,
        'entropy': entropy,
        'texture_contrast': glcm_c,
        'dom_R': dom_col[0],
        'dom_G': dom_col[1],
        'dom_B': dom_col[2]
    })

eda_df = pd.DataFrame(features)
eda_df.head()


| 🔢 Langkah                                         | 🎯 Tujuan / Apa yang Dicek                            | 🔍 Insight yang Didapat                                                                                        | ⚙️ Aksi / Tindakan yang Direkomendasikan                                                                                                                        |
| -------------------------------------------------- | ----------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **1. Distribusi Label (Bar Plot)**                 | Melihat keseimbangan jumlah gambar per kelas          | - Kelas tidak seimbang → potensi *class bias* saat training                                                    | - Tambahkan **augmentasi ekstra** pada kelas minoritas  <br> - Atau gunakan **class weights** di loss function                                                  |
| **2. Preview Sample Gambar per Kelas**             | Memverifikasi label benar, kualitas gambar normal     | - Ada label salah, gambar rusak, atau duplikasi                                                                | - Hapus / perbaiki gambar salah label  <br> - Bersihkan dataset sebelum training                                                                                |
| **3. Ukuran & Aspect Ratio Gambar**                | Mengecek variasi resolusi dan proporsi gambar         | - Gambar memiliki ukuran sangat bervariasi → model sulit generalisasi                                          | - Tentukan ukuran standar (`Resize(224,224)` atau `CenterCrop`)  <br> - Kalau perbedaan proporsi ekstrem → gunakan **padding atau random crop**                 |
| **4. Brightness (Rata-rata Luminance)**            | Mengecek pencahayaan antar kelas                      | - Kelas tertentu lebih terang/gelap dari lainnya  <br> - Model bisa bias ke pencahayaan                        | - Tambahkan augmentasi `ColorJitter` (brightness/contrast)  <br> - Gunakan histogram equalization (`cv2.equalizeHist`) sebelum training                         |
| **5. Contrast / Saturation**                       | Menilai seberapa kuat perbedaan warna dan pencahayaan | - Gambar low contrast → detail objek tidak terlihat jelas                                                      | - Gunakan `CLAHE` atau `Contrast Limited Adaptive Histogram Equalization` untuk memperbaiki  <br> - Tambahkan augmentasi `RandomContrast`                       |
| **6. Blur / Focus (Laplacian Variance)**           | Mengecek ketajaman / fokus gambar                     | - Beberapa gambar terlalu blur → noise tinggi, informasi rendah                                                | - Drop gambar dengan nilai focus < threshold  <br> - Atau augmentasi dengan blur acak (`GaussianBlur`) agar model robust                                        |
| **7. Edge Density (Canny)**                        | Mengukur seberapa kompleks bentuk dalam gambar        | - Kelas dengan sedikit edge → mungkin polos (sky, sea)  <br> - Banyak edge → tekstur padat (forest, buildings) | - Pastikan model cukup dalam untuk menangkap tekstur  <br> - Bisa gunakan CNN dengan kernel kecil (3×3) atau feature extractor pretrained                       |
| **8. Entropy (Informasi Visual)**                  | Mengukur banyaknya variasi piksel (detail informasi)  | - Entropy rendah → gambar terlalu polos (informasi sedikit)  <br> - Entropy tinggi → banyak tekstur / kompleks | - Gambar entropy rendah bisa dihapus atau diaugmentasi  <br> - Gunakan `RandomCrop` / `Noise Injection` untuk menambah variasi                                  |
| **9. Texture Contrast (GLCM)**                     | Mengevaluasi perbedaan tekstur antar kelas            | - Kelas dengan GLCM mirip → sulit dibedakan model                                                              | - Tambahkan augmentasi tekstur seperti `RandomErasing`, `CutMix`, atau `MixUp`                                                                                  |
| **10. Dominant Color (KMeans RGB)**                | Mengecek warna khas tiap kelas                        | - Kelas berbeda warna → mudah dibedakan model  <br> - Warna mirip → sulit dipisahkan                           | - Jika warna dominan terlalu mirip antar kelas → tambahkan augmentasi warna (hue shift)  <br> - Bisa pertimbangkan model yang lebih fokus ke bentuk (e.g., ViT) |
| **11. PCA / t-SNE Embedding (Feature Space)**      | Menilai *separabilitas visual* antar kelas            | - Cluster antar kelas terpisah → model mudah belajar  <br> - Cluster tumpang tindih → kelas visual mirip       | - Gunakan model lebih kompleks (e.g. ResNet50 → EfficientNet)  <br> - Atau gunakan augmentasi yang menonjolkan perbedaan antar kelas                            |
| **12. Cosine Similarity antar Centroid Fitur**     | Mengukur kemiripan antar kelas berdasarkan fitur      | - Similarity tinggi → kelas sering tertukar (misal mountain vs glacier)                                        | - Fokus augmentasi di kelas mirip  <br> - Analisis *confusion matrix* pasca-training untuk konfirmasi                                                           |
| **13. Warna & Histogram (cv2.calcHist)**           | Melihat distribusi piksel intensitas & channel warna  | - Distribusi warna sempit → data kurang variatif                                                               | - Tambah augmentasi warna (`Hue`, `Saturation`, `Brightness`)  <br> - Atau gunakan grayscale model bila warna tidak informatif                                  |
| **14. Outlier Check (Image Distance / Embedding)** | Cari gambar yang “tidak cocok” dengan kelasnya        | - Outlier bisa salah label atau anomali visual                                                                 | - Drop / re-label gambar tersebut agar model tidak bingung                                                                                                      |
| **15. Per Class Statistical Summary**              | Melihat rata-rata brightness, entropy, edge per kelas | - Ada kelas sangat kompleks, ada yang sederhana                                                                | - Gunakan augmentasi berbeda per kelas (kelas simple → augment kuat)                                                                                            |
